In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
sp500 = yf.Ticker("^GSPC")

In [ ]:
sp500 = sp500.history(period="max") 

In [ ]:
sp500

In [ ]:
del sp500["Dividends"]
del sp500["Stock Splits"]

In [ ]:
sp500

In [ ]:
sp500["Tomorrow Price"] = sp500["Close"].shift(-1)

In [ ]:
sp500

In [ ]:
sp500["Target"] = (sp500["Tomorrow Price"] > sp500["Close"]).astype(int)

In [ ]:
sp500

In [ ]:
sp500 = sp500.loc["1990-01-01":].copy()

In [ ]:
sp500

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=1)

train = sp500.iloc[:-100]
test = sp500.iloc[-100:]

predictors = ["Close", "Volume", "Open", "High", "Low"]
model.fit(train[predictors], train["Target"])

RandomForestClassifier(min_samples_split=100, random_state=1)

In [ ]:
from sklearn.metrics import precision_score

predictions = model.predict(test[predictors])

In [ ]:
import pandas as pd
predictions = pd.Series(predictions, index=test.index)

In [ ]:
precision_score(test["Target"], predictions)

0.3972602739726027

In [ ]:
def predict(train, test, predictors, model): 
    model.fit(train[predictors], train["Target"])
    predictions = model.predict(test[predictors])
    predictions = pd.Series(predictions, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], predictions], axis=1)
    return combined


In [ ]:
def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []

    for i in range(start, data.shape[0], step):
      train = data.iloc[0:i].copy()
      test = data.iloc[i:(i+step)].copy()
      predictions1 = predict(train, test, predictors, model)
      all_predictions.append(predictions1)
    return pd.concat(all_predictions)

In [ ]:
predictions1 = backtest(sp500, model, predictors)

In [ ]:
predictions1["Predictions"].value_counts()

0    3341
1    2435
Name: Predictions, dtype: int64

In [ ]:
precision_score(predictions1["Target"], predictions1["Predictions"])

0.5314168377823408

In [ ]:
predictions1["Target"].value_counts() / predictions1.shape[0]

1    0.534626
0    0.465374
Name: Target, dtype: float64

In [ ]:
horizons = [2,5,60,250,1000]
new_predictors = []

for horizon in horizons:
   rolling_averages = sp500.rolling(horizon).mean()

   ratio_column = f"Close_Ratio_{horizon}"
   sp500[ratio_column] = sp500["Close"] / rolling_averages["Close"]

   trend_column = f"Trend_{horizon}"
   sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()["Target"]

   new_predictors += [ratio_column, trend_column]

In [ ]:
sp500 = sp500.dropna()

In [ ]:
sp500

,Open,High,Low,Close,Volume,Tomorrow Price,Target,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_60,Trend_60,Close_Ratio_250,Trend_250,Close_Ratio_1000,Trend_1000
Date,,,,,,,,,,,,,,,,,
1993-12-14 00:00:00-05:00,465.730011,466.119995,462.459991,463.059998,275050000,461.839996,0,0.997157,1.0,0.996617,1.0,1.000283,32.0,1.028047,127.0,1.176082,512.0
1993-12-15 00:00:00-05:00,463.059998,463.690002,461.839996,461.839996,331770000,463.339996,1,0.998681,0.0,0.995899,1.0,0.997329,32.0,1.025151,126.0,1.172676,512.0
1993-12-16 00:00:00-05:00,461.859985,463.980011,461.859985,463.339996,284620000,466.380005,1,1.001621,1.0,0.999495,2.0,1.000311,32.0,1.028274,127.0,1.176163,513.0
1993-12-17 00:00:00-05:00,463.339996,466.380005,463.339996,466.380005,363750000,465.850006,0,1.003270,2.0,1.004991,3.0,1.006561,32.0,1.034781,128.0,1.183537,514.0
1993-12-20 00:00:00-05:00,466.380005,466.899994,465.529999,465.850006,255900000,465.299988,0,0.999431,1.0,1.003784,2.0,1.005120,32.0,1.033359,128.0,1.181856,513.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-27 00:00:00-04:00,3834.689941,3859.949951,3803.790039,3807.300049,4687320000,3901.060059,1,0.996949,0.0,0.999443,3.0,0.973664,23.0,0.900167,114.0,1.064469,543.0
2022-10-28 00:00:00-04:00,3808.260010,3905.419922,3808.260010,3901.060059,4459410000,3871.979980,0,1.012163,1.0,1.016144,3.0,0.998710,24.0,0.922957,114.0,1.090352,543.0
2022-10-31 00:00:00-04:00,3881.850098,3893.729980,3863.179932,3871.979980,4820620000,3856.100098,0,0.996259,1.0,1.004663,2.0,0.992422,24.0,0.916735,113.0,1.081902,543.0


In [ ]:
model = RandomForestClassifier(n_estimators=300, min_samples_split=50, random_state=1)

In [ ]:
def predict(train, test, predictors, model): 
    model.fit(train[predictors], train["Target"])
    predictions = model.predict_proba(test[predictors])[:,1]
    predictions[predictions >= .6] = 1
    predictions[predictions < .6] = 0
    predictions = pd.Series(predictions, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], predictions], axis=1)
    return combined

In [ ]:
predictions1 = backtest(sp500, model, new_predictors)

In [ ]:
predictions1["Predictions"].value_counts()

0.0    3993
1.0     782
Name: Predictions, dtype: int64

In [ ]:
precision_score(predictions1["Target"], predictions1["Predictions"])

0.5677749360613811